In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
import pickle

with open("cleaned_df_features.pkl", "rb") as f:
    df, numerics, curated_cat, other_cat = pickle.load(f)

In [3]:
# first we split trials

from sklearn.model_selection import train_test_split, KFold

# 10% for testing
[df_full_train,df_test] = train_test_split(df,test_size=0.1,random_state=42)
# 72% and 18% for train/val
[df_train,df_val] = train_test_split(df_full_train,test_size=0.2,random_state=42)
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(14514552, 13)
(3628639, 13)
(2015911, 13)


In [4]:
# data too large
del df
del df_full_train

import gc
gc.collect()

20

In [5]:
# get response variable out

y_test = df_test.base_passenger_fare.values
y_val = df_val.base_passenger_fare.values
y_train = df_train.base_passenger_fare.values

df_test = df_test.drop(columns=['base_passenger_fare'])
df_val = df_val.drop(columns=['base_passenger_fare'])
df_train = df_train.drop(columns=['base_passenger_fare'])

### starting with the curated feature set

In [6]:
# one-hot encoding: fit from training set

from sklearn.feature_extraction import DictVectorizer
import time

dv_cur = DictVectorizer(sparse=True)

# this takes some time
t0 = time.time()

# curated features only
X_cur_train = dv_cur.fit_transform(df_train[numerics + curated_cat].to_dict(orient='records'))
X_cur_val = dv_cur.transform(df_val[numerics + curated_cat].to_dict(orient='records'))

t1 = time.time()

print(f"DV took {(t1-t0)/60:5.2f} min")
print(X_cur_train.shape)

with open("dv_cur.pkl", "wb") as f:
    pickle.dump(dv_cur, f)

with open("X_cur_train.pkl", "wb") as f:
    pickle.dump(X_cur_train, f)

with open("X_cur_val.pkl", "wb") as f:
    pickle.dump(X_cur_val, f)
    
dv_cur.get_feature_names_out()

DV took  3.14 min
(14514552, 40)


array(['day_of_week=Friday', 'day_of_week=Monday', 'day_of_week=Saturday',
       'day_of_week=Sunday', 'day_of_week=Thursday',
       'day_of_week=Tuesday', 'day_of_week=Wednesday', 'hour_of_day=0',
       'hour_of_day=1', 'hour_of_day=10', 'hour_of_day=11',
       'hour_of_day=12', 'hour_of_day=13', 'hour_of_day=14',
       'hour_of_day=15', 'hour_of_day=16', 'hour_of_day=17',
       'hour_of_day=18', 'hour_of_day=19', 'hour_of_day=2',
       'hour_of_day=20', 'hour_of_day=21', 'hour_of_day=22',
       'hour_of_day=23', 'hour_of_day=3', 'hour_of_day=4',
       'hour_of_day=5', 'hour_of_day=6', 'hour_of_day=7', 'hour_of_day=8',
       'hour_of_day=9', 'hvfhs_license_num=Juno',
       'hvfhs_license_num=Lyft', 'hvfhs_license_num=Uber',
       'hvfhs_license_num=Via', 'shared_flag_or', 'trip_miles_log1p',
       'trip_time_log1p', 'wait_time_sec_log1p', 'wav_request_flag'],
      dtype=object)

In [14]:
ftns_cur = dv_cur.get_feature_names_out()
ftns_cur[[40-2,40-3,40-4]]

array(['wait_time_sec_log1p', 'trip_time_log1p', 'trip_miles_log1p'],
      dtype=object)

In [7]:
import psutil

mem = psutil.virtual_memory()
print(f"Total:     {mem.total/1e9:.2f} GB")
print(f"Available: {mem.available/1e9:.2f} GB")
print(f"Used:      {mem.used/1e9:.2f} GB")

Total:     16.88 GB
Available: 6.54 GB
Used:      10.34 GB


In [8]:
import pympler.asizeof as aso
print(aso.asizeof(dv_cur) / 1e9)
print(aso.asizeof(df_train) / 1e9)
size_bytes = (
    X_cur_train.data.nbytes +
    X_cur_train.indices.nbytes +
    X_cur_train.indptr.nbytes
)
print(size_bytes / 1e9)

5.576e-06
8.06498256
1.451455204


In [17]:


from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import Ridge, Lasso
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Optional: progress bar
from tqdm.auto import tqdm

print("\n||||||| curated features |||||||\n")

# ------------------------------------
# 1. Linear models: L2 (Ridge) search
# ------------------------------------

# Note: with one-hot/sparse features, use with_mean=False
# If X_* are sparse, this will keep them sparse where possible.


# only scale 3 numeric features
numeric_idx = [36,37,38]
all_idx = list(range(40))
cat_idx = [j for j in all_idx if j not in numeric_idx]

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(with_mean=False), numeric_idx),
        ("cat", "passthrough", cat_idx),
    ]
)


ridge_alphas = [0.0, 0.1, 1.0, 10.0, 100.0]

print("\n=== Ridge (L2) hyperparameter search ===")

ridge_results = []
for alpha in tqdm(ridge_alphas, desc="Ridge alphas"):
    model_ridge = Pipeline([
        ("scaler", preprocess),  # safe for sparse
        ("reg", Ridge(alpha=alpha, random_state=0))
    ])

    t0 = time.time()
    model_ridge.fit(X_cur_train, y_train)

    y_pred = model_ridge.predict(X_cur_train)
    rmse_train = root_mean_squared_error(y_train, y_pred)
    
    y_pred = model_ridge.predict(X_cur_val)
    rmse_val = root_mean_squared_error(y_val, y_pred)
    
    t1 = time.time()
    train_time = t1 - t0

    print(f"alpha={alpha:8.4f} | train_RMSE={rmse_train:8.4f} | val_RMSE={rmse_val:8.4f} | time={train_time/60:5.2f} min")
    ridge_results.append((alpha, rmse_train, rmse_val, train_time, model_ridge))

# Pick best Ridge by validation RMSE
best_ridge_alpha, best_ridge_rmse_train, best_ridge_rmse_val, best_ridge_time, best_ridge_model = min(
    ridge_results,
    key=lambda x: x[2]
)

print(f"\nBest Ridge alpha: {best_ridge_alpha}")
print(f"Best Ridge training RMSE: {best_ridge_rmse_train:.4f}")
print(f"Best Ridge validation RMSE: {best_ridge_rmse_val:.4f}")
print(f"Best Ridge training time: {best_ridge_time/60:.2f} minutes")


||||||| curated features |||||||


=== Ridge (L2) hyperparameter search ===


Ridge alphas:   0%|          | 0/5 [00:00<?, ?it/s]

alpha=  0.0000 | train_RMSE=  9.9316 | val_RMSE=  9.9607 | time= 0.32 min
alpha=  0.1000 | train_RMSE=  9.9316 | val_RMSE=  9.9607 | time= 0.31 min
alpha=  1.0000 | train_RMSE=  9.9316 | val_RMSE=  9.9607 | time= 0.30 min
alpha= 10.0000 | train_RMSE=  9.9316 | val_RMSE=  9.9607 | time= 0.31 min
alpha=100.0000 | train_RMSE=  9.9316 | val_RMSE=  9.9607 | time= 0.31 min

Best Ridge alpha: 0.1
Best Ridge training RMSE: 9.9316
Best Ridge validation RMSE: 9.9607
Best Ridge training time: 0.31 minutes


In [11]:
y_train_pred_baseline = np.full_like(y_train, np.mean(y_train), dtype=float)
y_val_pred_baseline = np.full_like(y_val, np.mean(y_train), dtype=float)

print("Baseline train RMSE:", root_mean_squared_error(y_train, y_train_pred_baseline))
print("Baseline valid RMSE:", root_mean_squared_error(y_val, y_val_pred_baseline))

Baseline train RMSE: 16.1098770146206
Baseline valid RMSE: 16.13452728439628


In [23]:
from sklearn.linear_model import SGDRegressor


# only scale 3 numeric features
numeric_idx = [36,37,38]
all_idx = list(range(40))
cat_idx = [j for j in all_idx if j not in numeric_idx]

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(with_mean=False), numeric_idx),
        ("cat", "passthrough", cat_idx),
    ]
)


SGDalphas = [1e-5, 1e-4, 1e-3, 1e-2]
l1_ratios = [0.0, 0.05, 0.1, 0.2, 0.5]

print("\n=== ElasticNet hyperparameter search ===")
SGD_results = []

for alpha in tqdm(SGDalphas, desc="SGD ElasticNet alphas"):
    for l1r in tqdm(l1_ratios, desc="l1 ratio", leave=False):
    
        model_SGD = Pipeline([
            ("scaler", preprocess),
            ("reg", SGDRegressor( \
                loss="squared_error",
                penalty="elasticnet",
                alpha=alpha,
                l1_ratio=l1r,
                eta0=1e-6,
                max_iter=100,
                shuffle=True,
                early_stopping=True,
                n_iter_no_change=5,
                validation_fraction=0.1,
                tol=1e-3,
                verbose=0)
            )
        ])
        
        t0 = time.time()
        model_SGD.fit(X_cur_train, y_train)
    
        y_pred = model_SGD.predict(X_cur_train)
        rmse_train = root_mean_squared_error(y_train, y_pred)
        
        y_pred = model_SGD.predict(X_cur_val)
        rmse_val = root_mean_squared_error(y_val, y_pred)
        
        t1 = time.time()
        train_time = t1 - t0
        
        print(f"alpha={alpha:8.6f} | l1r={l1r:8.4f} | train_RMSE={rmse_train:8.4f} | val_RMSE={rmse_val:8.4f} | time={train_time/60:5.2f} min")
        SGD_results.append((alpha, l1r, rmse_train, rmse_val, train_time, model_SGD))


with open("curated_SGDEN_results.pkl", "wb") as f:
    pickle.dump((SGD_results, l1_ratios, SGDalphas), f)

best_SGD_alpha, best_SGD_l1r, best_SGD_rmse_train, best_SGD_rmse_val, best_SGD_time, best_SGD_model = min(
    SGD_results,
    key=lambda x: x[3]
)

print(f"\nBest SGD alpha: {best_SGD_alpha}")
print(f"Best SGD L1 ratio: {best_SGD_l1r:.4f}")
print(f"Best SGD training RMSE: {best_SGD_rmse_train:.4f}")
print(f"Best SGD validation RMSE: {best_SGD_rmse_val:.4f}")
print(f"Best SGD training time: {best_SGD_time/60:.2f} minutes")


=== ElasticNet hyperparameter search ===


SGD ElasticNet alphas:   0%|          | 0/4 [00:00<?, ?it/s]

l1 ratio:   0%|          | 0/5 [00:00<?, ?it/s]

alpha=0.000010 | l1r=  0.0000 | train_RMSE= 10.2701 | val_RMSE= 10.2980 | time= 3.63 min
alpha=0.000010 | l1r=  0.0500 | train_RMSE= 10.2704 | val_RMSE= 10.2983 | time= 3.68 min
alpha=0.000010 | l1r=  0.1000 | train_RMSE= 10.2706 | val_RMSE= 10.2985 | time= 3.50 min
alpha=0.000010 | l1r=  0.2000 | train_RMSE= 10.2701 | val_RMSE= 10.2980 | time= 3.66 min
alpha=0.000010 | l1r=  0.5000 | train_RMSE= 10.2703 | val_RMSE= 10.2982 | time= 3.62 min


l1 ratio:   0%|          | 0/5 [00:00<?, ?it/s]

alpha=0.000100 | l1r=  0.0000 | train_RMSE= 10.2702 | val_RMSE= 10.2981 | time= 3.94 min
alpha=0.000100 | l1r=  0.0500 | train_RMSE= 10.2705 | val_RMSE= 10.2984 | time= 4.17 min
alpha=0.000100 | l1r=  0.1000 | train_RMSE= 10.2705 | val_RMSE= 10.2984 | time= 3.60 min
alpha=0.000100 | l1r=  0.2000 | train_RMSE= 10.2707 | val_RMSE= 10.2986 | time= 3.61 min
alpha=0.000100 | l1r=  0.5000 | train_RMSE= 10.2702 | val_RMSE= 10.2981 | time= 3.71 min


l1 ratio:   0%|          | 0/5 [00:00<?, ?it/s]

alpha=0.001000 | l1r=  0.0000 | train_RMSE= 10.2715 | val_RMSE= 10.2994 | time= 4.32 min
alpha=0.001000 | l1r=  0.0500 | train_RMSE= 10.2720 | val_RMSE= 10.2999 | time= 3.87 min
alpha=0.001000 | l1r=  0.1000 | train_RMSE= 10.2715 | val_RMSE= 10.2993 | time= 3.63 min
alpha=0.001000 | l1r=  0.2000 | train_RMSE= 10.2814 | val_RMSE= 10.3092 | time= 3.59 min
alpha=0.001000 | l1r=  0.5000 | train_RMSE= 10.2717 | val_RMSE= 10.2996 | time= 3.62 min


l1 ratio:   0%|          | 0/5 [00:00<?, ?it/s]

alpha=0.010000 | l1r=  0.0000 | train_RMSE= 10.3064 | val_RMSE= 10.3343 | time= 3.42 min
alpha=0.010000 | l1r=  0.0500 | train_RMSE= 10.2952 | val_RMSE= 10.3231 | time= 3.50 min
alpha=0.010000 | l1r=  0.1000 | train_RMSE= 10.3049 | val_RMSE= 10.3328 | time= 3.40 min
alpha=0.010000 | l1r=  0.2000 | train_RMSE= 10.2840 | val_RMSE= 10.3118 | time= 4.13 min
alpha=0.010000 | l1r=  0.5000 | train_RMSE= 10.2911 | val_RMSE= 10.3190 | time= 4.18 min

Best SGD alpha: 1e-05
Best SGD L1 ratio: 0.2000
Best SGD training RMSE: 10.2701
Best SGD validation RMSE: 10.2980
Best SGD training time: 3.66 minutes


### summary for Ridge and SGD Elastic Net regression with curated features

Ridge trains very fast (<1min) and SGD is a lot slower (a few minutes per model and more hyperparams). 

Both had similar performances, with ridge being slightly better (RMSE= \$9.96 vs \$10.30). 

(This is not what I expected. I think it might be because I did not pick the best hyperparams for SGD and did not let it train for very long.) 

Both training and validation RMSE are large, although they are better than the baseline model of mean (RMSE=\$16). 

Given the median fare is $10, the model is completely underfitting. 

Maybe the curated features are not informative enough, or the linear regression model is too biased, or maybe RMSE is not a good metric for training because of the skewness of our target variable.

In any case, I will try XGBoost next. Hopefully a more flexible model will give us better performance.

In [31]:
mem = psutil.virtual_memory()
print(f"Total:     {mem.total/1e9:.2f} GB")
print(f"Available: {mem.available/1e9:.2f} GB")
print(f"Used:      {mem.used/1e9:.2f} GB")

Total:     16.88 GB
Available: 2.60 GB
Used:      14.27 GB


In [30]:

#from xgboost import XGBRegressor
#from sklearn.model_selection import RandomizedSearchCV
#from scipy.stats import randint


# -------------------------------
# 3. XGBoost: randomized search
# -------------------------------

# portion out some data from _train for early stopping
X_temp, X_stop_xgb, y_temp, y_stop_xgb = train_test_split(
    X_cur_train, y_train, test_size=0.1, random_state=42)
# use fewer data for training for speed
X_temp, X_train_xgb, y_temp, y_train_xgb = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=42)

print(f"eval (early stopping) on {y_stop_xgb.shape[0]:d} rows")
print(f"train (CV tuning) on {y_train_xgb.shape[0]:d} rows")

# Define parameter distribution
param_dist = {
    'max_depth': [3,4,5,6,7,8],
    'learning_rate': [0.3, 0.2, 0.1, 0.05, 0.01],
    'min_child_weight': [1, 5, 10, 20],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Create XGBClassifier
xgb = XGBRegressor(
    tree_method="hist",
    enable_categorical=True,  # if using pandas categorical dtypes
    n_estimators=2000,        # large, rely on early stopping
    objective="reg:squarederror",
    eval_metric="rmse",
    early_stopping_rounds=10,
    n_jobs=-1
)

fit_params = {
    "eval_set": [(X_stop_xgb, y_stop_xgb)],
    "verbose": False,
}


search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=50,          # e.g. 50 random trials
    scoring="neg_root_mean_squared_error",
    verbose=2,          # shows progress of the search
    n_jobs=1,
    cv=3   
)

print("\n=== XGBoost hyperparameter search ===")
t0 = time.time()
search.fit(X_train_xgb, y_train_xgb, **fit_params)
t1 = time.time()




print("XGB Best params:", search.best_params_)
print("XGB Best CV score (RMSE):", -search.best_score_)
print(f"XGB search time: {(t1 - t0)/60:.2f} minutes")

best_xgb = search.best_estimator_

import pickle

with open("curated_xgboost_results.pkl", "wb") as f:
    pickle.dump((search, best_xgb, X_stop_xgb, y_stop_xgb, X_train_xgb, y_train_xgb), f)
    
# Evaluate best XGB on validation set explicitly:
y_pred_xgb = best_xgb.predict(X_cur_train)
xgb_train_rmse = root_mean_squared_error(y_train, y_pred_xgb)

y_pred_xgb = best_xgb.predict(X_cur_val)
xgb_val_rmse = root_mean_squared_error(y_val, y_pred_xgb)
print(f"XGB training RMSE (best model): {xgb_train_rmse:.4f}")
print(f"XGB validation RMSE (best model): {xgb_val_rmse:.4f}")



# ------------------------------------
# 4. Summary of model comparison
# ------------------------------------

#print("\n=== Summary (validation RMSE) ===")
#print(f"Ridge (L2)   : {best_ridge_rmse:.4f}  (alpha={best_ridge_alpha})")
#print(f"XGBoost      : {xgb_val_rmse:.4f}")

eval (early stopping) on 1451456 rows
train (CV tuning) on 2612620 rows

=== XGBoost hyperparameter search ===
Fitting 3 folds for each of 50 candidates, totalling 150 fits
XGB Best params: {'subsample': 1.0, 'min_child_weight': 10, 'max_depth': 8, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
XGB Best CV score (RMSE): 6.492647753591704
XGB search time: 379.53 minutes
XGB training RMSE (best model): 6.4728
XGB validation RMSE (best model): 6.5263


In [40]:
# let's check the results
results = search.cv_results_
df_results = pd.DataFrame({
    "mean_fit_time": results["mean_fit_time"],
    "param_subsample": results["param_subsample"],
    "param_min_child_weight": results["param_min_child_weight"],
    "param_max_depth": results["param_max_depth"],
    "param_learning_rate": results["param_learning_rate"],
    "param_colsample_bytree": results["param_colsample_bytree"],
    "mean_CV_RMSE": -results["mean_test_score"],
    "std_test_score": results["std_test_score"],
    "rank": results["rank_test_score"],
})

df_results.sort_values("rank",inplace=True)
df_results

,mean_fit_time,param_subsample,param_min_child_weight,param_max_depth,param_learning_rate,param_colsample_bytree,mean_CV_RMSE,std_test_score,rank
37,94.018815,1.0,10,8,0.05,0.6,6.492648,0.073968,1
21,101.153698,1.0,10,7,0.05,0.6,6.493559,0.072633,2
44,67.857698,1.0,5,5,0.20,0.6,6.495601,0.074099,3
49,600.627969,1.0,20,6,0.01,0.6,6.495954,0.068021,4
9,372.150743,1.0,1,7,0.01,0.6,6.496433,0.072570,5
12,79.833502,0.8,20,7,0.05,0.8,6.499159,0.068402,6
36,41.788510,1.0,10,6,0.20,0.6,6.500068,0.071850,7
47,196.109713,1.0,10,5,0.05,0.6,6.500827,0.074907,8
39,77.216035,0.6,1,7,0.05,0.6,6.502138,0.072641,9
3,40.063687,1.0,5,8,0.10,0.8,6.504243,0.072561,10


In [41]:
df_results["mean_CV_RMSE"].describe()

count    50.000000
mean      6.525098
std       0.027249
min       6.492648
25%       6.505818
50%       6.517779
75%       6.533978
max       6.594788
Name: mean_CV_RMSE, dtype: float64

### summary for XGBoost with curated features
XGBoost's RMSE is stuck very closely around $6.5 for both train and val. Hyperparameters make little difference to performance. This suggest underfitting. Because XGBoost is very powerful and flexible, I think the limiting factor is the curated features. It's time to use all features.